In [ ]:
import pandas as pd
import numpy as np
from statsmodels.discrete.discrete_model import Probit
from statsmodels.regression.mixed_linear_model import MixedLM
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sn
plt.style.use('default')
def standardization(data):
    data = np.array(data)
    mu = np.mean(data, axis=0)
    print(mu)
    sigma = np.std(data, axis=0)
    print(sigma)
    return (data ) / (sigma + 1e-9)


if __name__ == '__main__':

    conditions = ['RS','SR','RR','SS']
    const_list = []
    v_list = []
    std_const_list = []
    std_v_list = []
    # ori_list = [2.1702, 2.177, 2.0738, 2.5492,1.7513,1.852,1.7582,2.0644,1.1225,1.0966,1.122,1.511]
    ori_list = [0.0039548, 0.035103, 0.11163, -0.038259, -0.066627, -0.19945, -0.015113,0.025406,-0.063868,-0.014029,-0.0038405,-0.024054]
    ori_list_std = [0.026212,0.024205,0.047888,0.027847,0.021859,0.044628,0.022431,0.019441,0.031009,0.024677,0.020202,0.036573]
    fit_list = []
    fit_std_list = []
    colors = ['salmon', 'royalblue', 'goldenrod']
    for cond in conditions:
        path_list = [f'bandit_data/analysed_2D_fan_' + cond + 'high1.csv',f'bandit_data/analysed_2D_fan_'+cond+'low1.csv',f'bandit_data/analysed_2D_Gershman_'+cond+'4.csv']
        for path in path_list:
            Data = pd.read_csv(path)
            print(Data.shape)
            Data['choice'] = Data['choice'] & 1
            Data = Data[Data.trial > 4]
            Data = Data[Data.trial < 15]
            C_list = Data['choice'].tolist()
            Y = C_list
            V_list = (Data['Q0']-Data['Q1']).tolist()
            V_list = standardization(V_list)
            X = np.array(V_list).T
            X = pd.DataFrame(X, columns=['V'])
            X = sm.add_constant(X)
            model = Probit(Y, X)
            probit_model = model.fit()
            const = probit_model.params['const']
            v = probit_model.params['V']
            std_const = probit_model.params['const'] - probit_model.conf_int()[0]['const']
            std_v = probit_model.params['V'] - probit_model.conf_int()[0]['V']
            std_const_list.append(std_const)
            std_v_list.append(std_v)
            print(probit_model.summary())
            fit_list.append(const)
            fit_std_list.append(std_const)

    plt.errorbar(fit_list[::3], ori_list[::3], xerr=fit_std_list[::3], yerr=ori_list_std[::3], fmt='o', capsize=4,
                 capthick=2, color=colors[0], label='Fan23_high_anxiety')
    plt.errorbar(fit_list[1::3], ori_list[1::3], xerr=fit_std_list[1::3], yerr=ori_list_std[1::3], fmt='o', capsize=4,
                 capthick=2, color=colors[1], label='Fan23_low_anxiety')
    plt.errorbar(fit_list[2::3], ori_list[2::3], xerr=fit_std_list[2::3], yerr=ori_list_std[2::3], fmt='o', capsize=4,
                 capthick=2, color=colors[2], label='Gershman19')
    plt.ylabel('Human',size=20)
    plt.xlabel('Model',size=20)
    plt.ylim([-0.25,0.18])
    plt.xlim([-0.25, 0.18])
    
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    ax=plt.gca()
    locator = plt.MultipleLocator(0.1)
    ax.xaxis.set_major_locator(locator)
    plt.plot([-10, 10], [-10, 10], color='k', linestyle='--', linewidth=0.8)
    plt.title('intercept',size=20)
    plt.legend(prop={'size': 15})
    plt.show()
